# European Language Grid (ELG) Annotator

The `ElgAnnotator` is an annotator that onne of the  [ELG](https://live.european-language-grid.eu/catalogue/?&elg_compatible_service__term=Yes) services to annotate documents: for each document that gets processed, data is sent to a HTTP endpoint, processed there and information is sent back that is then used to annotate the document. 



In [1]:
from gatenlp import Document
# It is called "ElgTextAnnotator" because ELG also provides services to analyze audio and other kinds of data.
from gatenlp.processing.client import ElgTextAnnotator
from elg import Authentication

Lets try annotating a document with the UDPipe English: Morphosyntactic Analysis service 
(https://live.european-language-grid.eu/catalogue/tool-service/423). 

The service requires authentication, and there are several ways to get a key and provide the information to the ElgTextAnnotator class:

* `auth`: a pre-initialized ELG Authentication object
* `auth_file`: a json file that contains authentication information, generated with the ELG Authentication class
* `access_token`: this is the token that is shown on the service web page in the "Code samples" tab, under "cURL"
* `success_code`: the code created on the ELG web site for a logged in user when visiting one of these urls:
  * for short-term authentication that needs to get refreshed regularly: https://live.european-language-grid.eu/auth/realms/ELG/protocol/openid-connect/auth?client_id=elg-oob&redirect_uri=urn:ietf:wg:oauth:2.0:oob&response_type=code&scope=openid
  * for offline access: https://live.european-language-grid.eu/auth/realms/ELG/protocol/openid-connect/auth?client_id=elg-oob&redirect_uri=urn:ietf:wg:oauth:2.0:oob&response_type=code&scope=offline_access

To create and store authentication for offline access in a file `tokens.json` you can run the following code interactively (it will show the offline access URL from above and prompt for the success code): 
```
auth = Authentication.init(scope="offline_access")
auth.to_json("tokens.json")
print(f"The tokens will expire: {auth.refresh_expires_time}")
```
The following code assumes that a valid `tokens.json` file exists.

The service can be specified by:
* specify the ELG service nr using the `servicenr` parameter, e.g. `servicenr=423`
* specify the ELG service URL using the `url` parameter. The URL for the synchronous service is shown on the service page under the "Code samples" tab under "cURL"

Each service returns its own set of annotations and features. The ElgTextAnnotator constructor allows to specify a map via the `anntypes_map` to replace the original annotation types with new names when processing the document.

In [2]:
# auth = Authentication.init(scope="offline_access")
# auth.to_json("tokens.json")
# print(f"The tokens will expire: {auth.refresh_expires_time}")

In [6]:
doc = Document("Barack Obama visited Microsoft in New York last May.")
annt = ElgTextAnnotator(
    url="https://live.european-language-grid.eu/execution/process/udpipeen",
    auth_file="tokens.json", 
    anntypes_map = {"udpipe/paragraphs": "Paragraph", "udpipe/sentences": "Sentence", "udpipe/tokens": "UDPToken"}
)
doc = annt(doc)
doc

Document(Barack Obama visited Microsoft in New York last May.,features=Features({}),anns={'': AnnotationSet([Annotation(0,52,Paragraph,features=Features({}),id=0), Annotation(0,52,Sentence,features=Features({}),id=1), Annotation(0,6,UDPToken,features=Features({'words': [{'form': 'Barack', 'lemma': 'Barack', 'upos': 'PROPN', 'xpos': 'NNP', 'feats': 'Number=Sing', 'head': 3, 'deprel': 'nsubj'}]}),id=2), Annotation(7,12,UDPToken,features=Features({'words': [{'form': 'Obama', 'lemma': 'Obama', 'upos': 'PROPN', 'xpos': 'NNP', 'feats': 'Number=Sing', 'head': 1, 'deprel': 'flat'}]}),id=3), Annotation(13,20,UDPToken,features=Features({'words': [{'form': 'visited', 'lemma': 'visit', 'upos': 'VERB', 'xpos': 'VBD', 'feats': 'Mood=Ind|Tense=Past|VerbForm=Fin', 'head': 0, 'deprel': 'root'}]}),id=4), Annotation(21,30,UDPToken,features=Features({'words': [{'form': 'Microsoft', 'lemma': 'Microsoft', 'upos': 'PROPN', 'xpos': 'NNP', 'feats': 'Number=Sing', 'head': 3, 'deprel': 'obj'}]}),id=5), Annotation(31,33,UDPToken,features=Features({'words': [{'form': 'in', 'lemma': 'in', 'upos': 'ADP', 'xpos': 'IN', 'head': 7, 'deprel': 'case'}]}),id=6), Annotation(34,37,UDPToken,features=Features({'words': [{'form': 'New', 'lemma': 'New', 'upos': 'PROPN', 'xpos': 'NNP', 'feats': 'Number=Sing', 'head': 7, 'deprel': 'compound'}]}),id=7), Annotation(38,42,UDPToken,features=Features({'words': [{'form': 'York', 'lemma': 'York', 'upos': 'PROPN', 'xpos': 'NNP', 'feats': 'Number=Sing', 'head': 3, 'deprel': 'obl'}]}),id=8), Annotation(43,47,UDPToken,features=Features({'words': [{'form': 'last', 'lemma': 'last', 'upos': 'ADJ', 'xpos': 'JJ', 'feats': 'Degree=Pos', 'head': 9, 'deprel': 'amod'}]}),id=9), Annotation(48,51,UDPToken,features=Features({'words': [{'form': 'May', 'lemma': 'May', 'upos': 'PROPN', 'xpos': 'NNP', 'feats': 'Number=Sing', 'head': 3, 'deprel': 'obl:tmod'}]}),id=10), Annotation(51,52,UDPToken,features=Features({'words': [{'form': '.', 'lemma': '.', 'upos': 'PUNCT', 'xpos': '.', 'head': 3, 'deprel': 'punct'}]}),id=11)])})

UDPToken annotations are not very useful as they are, because each token contains a single feature describing one or more words corresponding to these tokens (UDPipe services support multi-word tokens). 

To convert these tokens to token annotations (for single word tokens) and MWT (multi word token) and Token annotations for multi word tokens the following utility method can be used. This method will also adapt the dependency parser ids to refer to the respective Token annotation ids (the head id is always mapped to the containing sentence annotation) and will split the "feats" feature up into separate features on the annotation.

In [7]:
from gatenlp.processing.client.elg import udptoken2tokens
udptoken2tokens(doc.annset().with_type("UDPToken"), doc.annset().with_type("Sentence"), doc.annset())
doc

Document(Barack Obama visited Microsoft in New York last May.,features=Features({}),anns={'': AnnotationSet([Annotation(0,52,Paragraph,features=Features({}),id=0), Annotation(0,52,Sentence,features=Features({}),id=1), Annotation(0,6,UDPToken,features=Features({'words': [{'form': 'Barack', 'lemma': 'Barack', 'upos': 'PROPN', 'xpos': 'NNP', 'feats': 'Number=Sing', 'head': 3, 'deprel': 'nsubj'}]}),id=2), Annotation(0,6,Token,features=Features({'form': 'Barack', 'lemma': 'Barack', 'upos': 'PROPN', 'xpos': 'NNP', 'head': 14, 'deprel': 'nsubj', 'Number': 'Sing'}),id=12), Annotation(7,12,UDPToken,features=Features({'words': [{'form': 'Obama', 'lemma': 'Obama', 'upos': 'PROPN', 'xpos': 'NNP', 'feats': 'Number=Sing', 'head': 1, 'deprel': 'flat'}]}),id=3), Annotation(7,12,Token,features=Features({'form': 'Obama', 'lemma': 'Obama', 'upos': 'PROPN', 'xpos': 'NNP', 'head': 12, 'deprel': 'flat', 'Number': 'Sing'}),id=13), Annotation(13,20,UDPToken,features=Features({'words': [{'form': 'visited', 'lemma': 'visit', 'upos': 'VERB', 'xpos': 'VBD', 'feats': 'Mood=Ind|Tense=Past|VerbForm=Fin', 'head': 0, 'deprel': 'root'}]}),id=4), Annotation(13,20,Token,features=Features({'form': 'visited', 'lemma': 'visit', 'upos': 'VERB', 'xpos': 'VBD', 'head': 1, 'deprel': 'root', 'Mood': 'Ind', 'Tense': 'Past', 'VerbForm': 'Fin'}),id=14), Annotation(21,30,UDPToken,features=Features({'words': [{'form': 'Microsoft', 'lemma': 'Microsoft', 'upos': 'PROPN', 'xpos': 'NNP', 'feats': 'Number=Sing', 'head': 3, 'deprel': 'obj'}]}),id=5), Annotation(21,30,Token,features=Features({'form': 'Microsoft', 'lemma': 'Microsoft', 'upos': 'PROPN', 'xpos': 'NNP', 'head': 14, 'deprel': 'obj', 'Number': 'Sing'}),id=15), Annotation(31,33,UDPToken,features=Features({'words': [{'form': 'in', 'lemma': 'in', 'upos': 'ADP', 'xpos': 'IN', 'head': 7, 'deprel': 'case'}]}),id=6), Annotation(31,33,Token,features=Features({'form': 'in', 'lemma': 'in', 'upos': 'ADP', 'xpos': 'IN', 'head': 18, 'deprel': 'case'}),id=16), Annotation(34,37,UDPToken,features=Features({'words': [{'form': 'New', 'lemma': 'New', 'upos': 'PROPN', 'xpos': 'NNP', 'feats': 'Number=Sing', 'head': 7, 'deprel': 'compound'}]}),id=7), Annotation(34,37,Token,features=Features({'form': 'New', 'lemma': 'New', 'upos': 'PROPN', 'xpos': 'NNP', 'head': 18, 'deprel': 'compound', 'Number': 'Sing'}),id=17), Annotation(38,42,UDPToken,features=Features({'words': [{'form': 'York', 'lemma': 'York', 'upos': 'PROPN', 'xpos': 'NNP', 'feats': 'Number=Sing', 'head': 3, 'deprel': 'obl'}]}),id=8), Annotation(38,42,Token,features=Features({'form': 'York', 'lemma': 'York', 'upos': 'PROPN', 'xpos': 'NNP', 'head': 14, 'deprel': 'obl', 'Number': 'Sing'}),id=18), Annotation(43,47,UDPToken,features=Features({'words': [{'form': 'last', 'lemma': 'last', 'upos': 'ADJ', 'xpos': 'JJ', 'feats': 'Degree=Pos', 'head': 9, 'deprel': 'amod'}]}),id=9), Annotation(43,47,Token,features=Features({'form': 'last', 'lemma': 'last', 'upos': 'ADJ', 'xpos': 'JJ', 'head': 20, 'deprel': 'amod', 'Degree': 'Pos'}),id=19), Annotation(48,51,UDPToken,features=Features({'words': [{'form': 'May', 'lemma': 'May', 'upos': 'PROPN', 'xpos': 'NNP', 'feats': 'Number=Sing', 'head': 3, 'deprel': 'obl:tmod'}]}),id=10), Annotation(48,51,Token,features=Features({'form': 'May', 'lemma': 'May', 'upos': 'PROPN', 'xpos': 'NNP', 'head': 14, 'deprel': 'obl:tmod', 'Number': 'Sing'}),id=20), Annotation(51,52,UDPToken,features=Features({'words': [{'form': '.', 'lemma': '.', 'upos': 'PUNCT', 'xpos': '.', 'head': 3, 'deprel': 'punct'}]}),id=11), Annotation(51,52,Token,features=Features({'form': '.', 'lemma': '.', 'upos': 'PUNCT', 'xpos': '.', 'head': 14, 'deprel': 'punct'}),id=21)])})